In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter 
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_ollama import OllamaLLM
from langchain import PromptTemplate
from langchain_community.document_loaders import TextLoader
from langchain.schema import Document
from langgraph.prebuilt import create_react_agent
import datetime
import numpy as np
import time
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder


In [2]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [3]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH00-RAG-Chatbot")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH00-RAG-Chatbot


In [ ]:
OLLAMA_BASE_URL = "http://localhost:11434" # Ollama 서버 주소
OLLAMA_CHAT_MODEL = "ollama-ko-0710:latest" # Ollama에서 실행중인 모델 이름

## ReRank 추가 버전

In [4]:
def general_llm():
    # 3. 텍스트 청크 분할 (chunk_size=1000, chunk_overlap=50)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    loader = TextLoader("./data/text.txt", encoding="utf-8")
    split_docs = loader.load_and_split(text_splitter)
    model_name = "intfloat/multilingual-e5-large-instruct"
    hf_embeddings = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs={"device": "cuda"},  # cuda, cpu
        encode_kwargs={"normalize_embeddings": True},
    )

    db = FAISS.from_documents(documents=split_docs, embedding=hf_embeddings)

    # 5. 리트리버 생성
    retriever = db.as_retriever(
        search_type="mmr", search_kwargs={"k": 10, "lambda_mult": 0.25, "fetch_k": 20}
    )

    reranker = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")
    compressor = CrossEncoderReranker(model=reranker, top_n=6)
    compression_retriever = ContextualCompressionRetriever(
        base_compressor=compressor, base_retriever=retriever
    )
    # compressed_docs = compression_retriever.invoke("삼성전자 하반기 실적 전망은?")
    # pretty_print_docs(compressed_docs)

    template = """당신은 유용한 AI 어시스턴트입니다. 사용자의 질의에 대해 친절하고 정확하게 답변해야 합니다.
    You are a helpful AI assistant, you'll need to answer users' queries in a friendly and accurate manner.
    모든 대답은 반드시 한국말로 대답해주세요.
    
    문서 내용:
    {context}
    질문: {question}
    답변:"""
    prompt = PromptTemplate(template=template, input_variables=["context", "question"])
    
    # # 7. Ollama LLM 초기화
    # llm = OllamaLLM(model=OLLAMA_CHAT_MODEL, base_url=OLLAMA_BASE_URL,temperature=1, max_tokens=1024)
    llm = ChatOpenAI(model_name="gpt-4.1-mini", temperature=0.7, max_tokens=1024)

    # # 8. RAG 체인 생성
    chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=compression_retriever,
        return_source_documents=True,
        chain_type_kwargs={"prompt": prompt}
    )

    result = chain.invoke({"query": "삼성전자 하반기 실적 전망은?"})
    answer = result["result"]
    return answer

In [ ]:
general_llm()



'삼성전자의 2023년 하반기 실적 전망은 긍정적입니다. 3분기 영업이익은 약 9조 원으로, 전년 동기 대비 1.9% 감소하나 직전 분기 대비 91.9% 증가할 것으로 예상됩니다. 또한, 미래에셋증권은 삼성전자의 올해와 내년 영업이익 전망치를 각각 2.9%, 5.3% 상향 조정했고, 목표주가도 8만8000원에서 9만6000원으로 9% 높였습니다. 이는 내년 메모리 반도체 공급 부족과 제한된 D램 생산라인, 차세대 반도체(HBM) 증설 집중 덕분에 범용 D램 가격이 오를 것으로 전망되기 때문입니다. 전반적으로 중장기 경쟁력 회복과 밸류에이션 정상화에 대한 기대감이 커지고 있습니다.'

: 